In [62]:
# !pip install numpy-financial

## Imports

In [63]:
# Read stocks
import yfinance as yf

# To get analysts info
import yahoo_fin.stock_info as si

# Required to calculate net present value
import numpy_financial as npf

# For reading properties
from jproperties import Properties

## Load properties

In [64]:
configs = Properties()

with open('config/yf_dcf.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
# Perpetual growth rates is read as a string; we need to convert it to float
PERP_GROWTH_RATE = float(configs.get('PERP_GROWTH_RATE').data)
WACC_RATE = float(configs.get('WACC_RATE').data)

## Constants

In [65]:
PROJECTION_YEARS = 5

## Get Growth Rate

In [66]:
growth_est_df = si.get_analysts_info(TICKER)['Growth Estimates']
growth_str = growth_est_df[growth_est_df['Growth Estimates'] == 'Next 5 Years (per annum)'][TICKER].iloc[0]
growth_rate = round(float(growth_str.rstrip('%')) / 100.0, 4)

## Future Free Cash Flow

In [67]:
# Using yfinance to get other data
ticker = yf.Ticker(TICKER)
	
# Free cash flow
free_cash_flow = ticker.cash_flow.loc['Free Cash Flow'][0]

ffcf = []
# Year 1
ffcf.append(free_cash_flow * (1 + growth_rate))

# Starting from year 2
for i in range(1, PROJECTION_YEARS):
    ffcf.append(ffcf[i-1] * (1 + growth_rate))
ffcf

[118530774800.00002,
 126069332077.28003,
 134087341597.39505,
 142615296522.98938,
 151685629381.85153]

## Calculate the Terminal Value

In [68]:
# forecast_fcf[-1] refers to the last year in the growth period
terminal_value = ffcf[-1] * (1 + PERP_GROWTH_RATE)/(WACC_RATE - PERP_GROWTH_RATE)

## Enterprise Valuation

In [69]:
# Add the terminal value to the last year
ffcf[-1] = ffcf[-1] + terminal_value

# Calcualte the enterprise_value using npv - add zero or else the method assumes
# first value as the initial investment
enterprise_value = npf.npv(WACC_RATE, [0] + ffcf)
# enterprise_value = sum(pv_ffcf)

## Equity Valuation

In [70]:
# Calculate Cash And Cash Equivalents
balance_sheet_df = ticker.balance_sheet
cash_and_equivalents = balance_sheet_df.loc['Cash And Cash Equivalents'][0]

# Current debt - only interested in the value
current_debt = balance_sheet_df.loc['Current Debt'][0]

# Long term debt - only interested in the value
long_term_debt = balance_sheet_df.loc['Long Term Debt'][0]

# Equity value -> enterprise_value + cash - total debt
equity_value = enterprise_value + cash_and_equivalents - (current_debt +  long_term_debt)

## Intrinsic Valuation

In [71]:
# Shares outstanding
shares_outstanding = ticker.info['sharesOutstanding']

# Current stock price
current_value = ticker.info['currentPrice']

# Intrinsic value
intrinsic_value = equity_value / shares_outstanding
'Current Price: ${:.2f} Intrinsic Value: ${:.2f}'.format(round(current_value,2), round(intrinsic_value,2)) + ' for ticker ' + TICKER

'Current Price: $174.49 Intrinsic Value: $117.87 for ticker AAPL'

In [72]:
upside = (intrinsic_value / current_value) - 1
'{:.2%}'.format(round(upside,4)) + ' for ticker ' + TICKER

'-32.45% for ticker AAPL'

In [73]:
if intrinsic_value > current_value:
    print('BUY')
else:
    print('SELL')

SELL


## Enterprise Valuation - alternative

In [78]:
# This is another way of calculating PV of FFCF - sum of it should match the npv calculation of npf
pv_ffcf = []
for idx, x in enumerate(ffcf):
    pv_ffcf.append(x/(1 + WACC_RATE)**(idx+1))
# This is equivalent to enterprise_value value
sum(pv_ffcf)

1950361999312.4448